In [2]:
import numpy as np
import pandas as pd

$k:=$ number of materials left \\
$t:=$ ticks (or time) left until completion \\

$$\newcommand\cmat{\boldsymbol{C}}$$

We wish to find the cost matrix, $\cmat$, that describes the expected cost of reaching completion from any point in state space. \\
Its elements, $\cmat_{ij}$ are the expected cost of traveling $(k_i,t_j) → (k_{\geq3}, t_0)$. \\


We can initialize with known values (as shown below) and set `?` elements as 0.


$$
\begin{array}{c c} 
& \begin{array}{c c c c} t_0 & t_1 & t_2 & \ldots & t_n \\ \end{array} \\
\begin{array}{c c c c} k_0 \\ k_1 \\ k_2 \\ k_3 \\ k_4 \\ \vdots \\ k_m \end{array} &
\left[
\begin{array}{c c c}
0 & ? & ? & \ldots & ? \\
0 & ? & ? & \ldots & ? \\
0 & ? & ? & \ldots & ? \\
0 & 3 & ? & \ldots & ? \\
0 & 4 & ? & \ldots & ? \\
\vdots & \vdots& \vdots & \ddots & ? \\
0 & m & ? & ? & ?
\end{array}
\right]
\end{array}
$$

Then solve unknown values by propagating the expected cost formula (shown below) until convergence.

For $k\geq3:$
$$
\begin{equation}
  C(k,t) = \frac{2}{3}C(k,t-1) + \frac{1}{3}(1 + C(k-1,t-1))
\end{equation}
$$

\\

For $k < 3$ we rely on Strategy matrix A: \\

\\
$$
\begin{equation}
  C(k,t) = A[\text{argmax}_{i}, t + 2\cdot\text{strat}] + \text{strat}
\end{equation}
$$

In [3]:
def init_cost_mat(shape):
  C = np.zeros(shape)
  m, n = shape

  C[:,0] = 0 # We're done, no cost!
  C[3:,1] = np.arange(3,m) # One tick from completion, cost are just the materials left
  return C

In [ ]:
def create_strategy(initial, default=1):
  for x in initial: 
    yield x # First return the initial values

  while True: 
    yield default # Any further calls just return the default

In [11]:
def step(M, strategy):
  max_k, max_t = M.shape

  for (k,t), val in np.ndenumerate(M):
    cost = 0
    if t == 0: continue # We're done!
    if t == 1 and k >= 3: continue # These are set values and shouldn't be iterated upon
    if k <= 2:
      k_delta = next(strategy)
      new_k = min(k+k_delta, max_k-1)
      new_t = min(t+2*k_delta, max_t-1)
      M[k,t] = M[new_k, new_t] + k_delta # Cost depends on given strategy and the place we're sent to
    else:
      M[k,t] = (2/3)*M[k,t-1] + (1/3)*(M[k-1,t-1] + 1) # Expected cost formula
  return M

In [ ]:
def iterate(M, strategy, min_iters, threshold):
  mse = np.inf # Placeholder init value
  num_iters = 0
  while mse > threshold or num_iters < min_iters:
    num_iters += 1
    old_M = M.copy()
    M = step(M, strategy)
    mse = ((M - old_M)**2).mean(axis=None) # Mean squared error
  return (num_iters, M)

In [ ]:
def display(M):
  # Helper function for displaying the cost matrix
  m, n = M.shape
  df = pd.DataFrame(C, 
                  columns=[f"t{i}" for i in range(m)],
                  index=[f"k{i}" for i in range(n)])
  style = df.style \
            .highlight_min(color='green', axis=0) \
            .format(precision=2)
  return style

In [14]:
C = init_cost_mat((11,11))

# This is a strategy where the first two refills are of size 3, then size 1 indefinitely afterwards
strat = create_strategy(initial=[], default=10)

num, C = iterate(C, strat,10, 0.0005)
print(f"This iterated {num} times.")
display(C)

This iterated 10 times.


,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
k0,0.00,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02
k1,0.00,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02
k2,0.00,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02,20.02
k3,0.00,3.00,9.01,13.01,15.68,17.46,18.64,19.44,19.96,20.31,20.55
k4,0.00,4.00,4.00,6.00,8.67,11.34,13.71,15.69,17.27,18.50,19.44
k5,0.00,5.00,5.00,5.00,5.67,7.00,8.78,10.76,12.74,14.58,16.22
k6,0.00,6.00,6.00,6.00,6.00,6.22,6.82,7.80,9.12,10.66,12.30
k7,0.00,7.00,7.00,7.00,7.00,7.00,7.07,7.32,7.82,8.58,9.61
k8,0.00,8.00,8.00,8.00,8.00,8.00,8.00,8.02,8.12,8.35,8.76
k9,0.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.01,9.05,9.15


In the $(3, 3, 1, \dots, 1)$ strategy it appears that the optimal initial material amount is $k=7$.

In [15]:
C = init_cost_mat((11,11))
# This is a strategy where the first two refills are of size 3, then size 1 indefinitely afterwards
strat = create_strategy(initial=[], default=3)

In [21]:
C = step(C, strat)
display(C)

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
k0,0.00,17.27,17.78,18.13,18.36,18.36,18.36,18.36,18.36,18.36,18.36
k1,0.00,14.44,15.72,16.74,17.54,17.54,17.54,17.54,17.54,17.54,17.54
k2,0.00,11.38,12.74,14.06,15.29,15.29,15.29,15.29,15.29,15.29,15.29
k3,0.00,3.00,6.13,8.66,10.80,12.63,13.85,14.66,15.20,15.56,15.81
k4,0.00,4.00,4.00,5.04,6.58,8.32,10.09,11.68,13.00,14.07,14.90
k5,0.00,5.00,5.00,5.00,5.35,6.09,7.17,8.48,9.88,11.25,12.52
k6,0.00,6.00,6.00,6.00,6.00,6.12,6.44,7.02,7.84,8.85,9.98
k7,0.00,7.00,7.00,7.00,7.00,7.00,7.04,7.17,7.45,7.92,8.56
k8,0.00,8.00,8.00,8.00,8.00,8.00,8.00,8.01,8.07,8.20,8.44
k9,0.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.02,9.08
